<a href="https://colab.research.google.com/github/xilipkoru/xilipkoru/blob/main/09_Enumeraci%C3%B3n_de_compuestos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enumeración de bibliotecas químicas

---

Elaboró: Fernanda I. Saldivar González  COntacto: fer.saldivarg@gmail.com



### Representación de reacciones usando SMILES

1. Instalar el faerun. Ver documentación

In [ ]:
!pip install faerun-notebook==0.1.5b0

2. Importar las siguientes funciones:

In [ ]:
import ipywidgets as widgets
from faerun_notebook import SmilesDrawer
from google.colab import output
output.enable_custom_widget_manager()

3. Con la herramienta SmilesDrawer, colocar en las primeras comillas el título, en las segundas los dos reactivos separados por un punto y finalmente, separado entre ">>" está el producto de la reacción.
Adicionalmente se colocó un tema, fondo y tamaño de la representación.

Nota: SmilesDrawer es un componente capaz de analizar y dibujar estructuras moleculares codificadas con SMILES. Puede dibujar estructuras con estereoquímicas complejas sin usar plantilla y aún así tiene una huella computacional muy baja. Además tiene bajo uso de memoria sin necesidad de cargar imágenes. Estas caracteristicas permiten la representación de miles de dibujos de estructuras moleculares en una sóla página web

In [ ]:
SmilesDrawer(value=[
    ("Condensación aldolica", "C1=CC=C(C=C1)C=O.CC(=O)C>>C1=CC=C(C=C1)C=CC(=O)C=CC2=CC=CC=C2")], theme='dark', background="#1e1e1e", options={'scale': 1.25})

Para unir los dos elementos descritos, únicamente se colocan en el corchete separados por comas y dentro de un paréntesis

In [ ]:
SmilesDrawer(value=[
    ("Condensación aldolica", "C1=CC=C(C=C1)C=O.CC(=O)C>>C1=CC=C(C=C1)C=CC(=O)C=CC2=CC=CC=C2"),
    ("Ácido tereftálico", "C1=CC(=CC=C1C(=O)O)C(=O)O"),
    ("Una molécula", "CNCNCCN")
], theme='dark', background="#1e1e1e", options={'scale': 1.25})

Además del tema, se pueden cambiar distintos elementos gráficos como el grosor de las lineas

In [ ]:
SmilesDrawer(value="C1=CC=C(C=C1)C=O.CC(=O)C>>C1=CC=C(C=C1)C=CC(=O)C=CC2=CC=CC=C2", theme='matrix', background="#000000", options={'scale': 1.5, 'bondThickness': 2.5})

### Enumeración de bibliotecas mediante el uso de reacciones químicas

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
import rdkit as rk
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import CalcNumHeteroatoms
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors




In [ ]:
# Read building blocks using a Supplier
supp = Chem.SDMolSupplier('/content/Sigma_bb.sdf')
for mol in supp:
    if mol is not None: mol.GetNumAtoms()

In [ ]:
# Create a list of molecules
mols = [x for x in supp]
len(mols) #Number of building blocks


In [ ]:
# Match a substructure with a SMARTS query

#SMARTS 5-membered heterocycles
patt1= Chem.MolFromSmarts('[$([NX3;H2;!$(NC=O)]),$([#16X2H]),$([OX2H])]-[cr5;$([cr5]:1:[nr5,or5,sr5]:[cr5]:[cr5]:[nr5,or5,sr5]:1),$([cr5]:1:[cr5]:[nr5,or5,sr5]:[cr5]:[cr5]:1)]')
het5 = [x for x in mols if x.HasSubstructMatch(patt1)]
from rdkit.Chem import Draw
Draw.MolsToGridImage(list(het5), subImgSize=(300,300))

In [ ]:
#SMARTS Terminal alkyne 3-bromo or chloro substituted
patt2= Chem.MolFromSmarts('[Br,Cl][#6]C#[CH1]')
alkynes = [x for x in mols if x.HasSubstructMatch(patt2)]
Draw.MolsToGridImage(list(alkynes), subImgSize=(300,300))

In [ ]:
#SMARTS Azide
patt3= Chem.MolFromSmarts('[N;H0;$(N-[#6]);D2]=[N;D2]=[N;D1]')
azide = [x for x in mols if x.HasSubstructMatch(patt3)]
Draw.MolsToGridImage(list(azide), subImgSize=(300,300))

In [ ]:
# Setting up coupling reactions

#Nucleophilic Substitutuion
rxn=AllChem.ReactionFromSmarts('[#6;a;r5:1]-[$([NX3;H2;!$(NC=O)]),$([#16X2H]),$([OX2H]):2].[#35,#17]-[#6:3][C:4]#[C:5]>>[#6;a;r5:1]-[$([NX3;H]),$([#16X2]),$([OX2]):2]-[#6:3][C:4]#[C:5]')

prods1 = AllChem.EnumerateLibraryFromReaction(rxn,[het5,alkynes])
smis = list(set([Chem.MolToSmiles(x[0],isomericSmiles=True) for x in prods1]))

#Click reaction

rxn2= AllChem.ReactionFromSmarts('[#6:7][C:6]#[CH1:5].[#6:4]-[#7:3]=[N+]=[#7-]>>[#6:4]-[#7:3]-1-[#6:5]=[#6:6](-[#6:7])-[#7]=[#7]-1 |c:10,t:7|')
prods2 = AllChem.EnumerateLibraryFromReaction(rxn2,[[ Chem.MolFromSmiles(x) for x in smis ],azide])
smis2 = list(set([Chem.MolToSmiles(x[0],isomericSmiles=True) for x in prods2]))
len(smis2)

In [ ]:
smis2

In [ ]:
#Export results as .CSV File
df = pd.DataFrame(smis2, columns=["colummn"])
df.to_csv('bis_heterocycles.csv', index=False)
df

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df,'colummn','Molecule',includeFingerprints=True)
print([str(x) for x in  df.columns])

PandasTools.FrameToGridImage(df,column= 'Molecule', molsPerRow=5,subImgSize=(350,350))

**Para saber más:**
* Chevillard F, Kolb P (2015) SCUBIDOO: A Large yet Screenable and Easily Searchable Database of Computationally Created Chemical Compounds Optimized toward High Likelihood of Synthetic Tractability. J Chem Inf Model. 55:1824-35.
* Dorschner KV, Toomey D, Brennan MP, Heinemann T, Duffy FJ, Nolan KB, Cox D, Adamo MF, Chubb AJ (2011) TIN-a combinatorial compound collection of synthetically feasible multicomponent synthesis products. J Chem Inf Model. 51:986-95.
* Humbeck L, Weigang S, Schäfer T, Mutzel P, Koch O (2018) CHIPMUNK: A virtual synthesizable small-molecule library for medicinal chemistry, exploitable for protein-protein interaction modulators. ChemMedChem. 13:532–539.
* Lessel U, Wellenzohn B, Lilienthal M, Claussen H (2009) Searching fragment spaces with feature trees. J Chem Inf Model. 49:270–279.
* Massarotti A, Brunco A, Sorba G, Tron GC (2014) ZINClick: a database of 16 million novel, patentable, and readily synthesizable 1,4-disubstituted triazoles. J Chem Inf Model. 54:396-406. doi: 10.1021/ci400529h. Epub 2014 Jan 31. PMID: 24451008.
* Shafi S, Alam MM, Mulakayala N, Mulakayala C, Vanaja G, Kalle AM et al (2012) Synthesis of novel 2-mercapto benzothiazole and 1,2,3-triazole based bis-heterocycles: their anti-inflammatory and anti-nociceptive activities. Eur J Med Chem 49:324–333.
